In [1]:
from fftw import generate_fft, default_config
import numpy as np

In [2]:
default_config["generator_path"] = "../genfft"
default_config["compile_args"] = ["-x", "c++", "-O3", "-include", "fftw/codelet.hh", "-shared"]

In [57]:
fft16 = generate_fft(default_config, "notw", n=16)
fft4 = generate_fft(default_config, "notw", n=4)
fft5 = generate_fft(default_config, "notw", n=5)
fft3 = generate_fft(default_config, "notw", n=3)
fft2 = generate_fft(default_config, "notw", n=2)
fft6 = generate_fft(default_config, "notw", n=6)

twiddle2 = generate_fft(default_config, "twiddle", n=2)
twiddle4 = generate_fft(default_config, "twiddle", n=4)
twiddle3 = generate_fft(default_config, "twiddle", n=3)
twiddle5 = generate_fft(default_config, "twiddle", n=5)


def w(k, n):
    return np.exp(2j * np.pi * k / n)


def make_twiddle(n1, n2):
    I1 = np.arange(n1)
    I2 = np.arange(n2)
    return w(I1[:,None] * I2[None,:], n1*n2).astype('complex64')


def fft_two_factor(config, n, m):
    fft_n = generate_fft(config, "notw", n=n)
    fft_m = generate_fft(config, "notw", n=m)
    W = make_twiddle(m, n).conj()
    
    def fft(x):
        y = np.zeros_like(x).reshape([m, n])
        z = np.zeros_like(x)
        fft_n(x.reshape([n, m]).T, y)
        y *= W
        fft_m(y.T, z.reshape([m, n]).T)
        return z
    
    return fft


fft_p12 = fft_two_factor(default_config, 3, 4)
fft_p12i = fft_two_factor(default_config, 6, 2)


def fft_two_factor_twiddle(config, n, m):
    fft_n = generate_fft(config, "notw", n=n)
    fft_m = generate_fft(config, "twiddle", n=m)
    W = make_twiddle(n, m)[:,1:].copy()
    
    def fft(x):
        y = np.zeros_like(x).reshape([m, n])
        fft_n(x.reshape([n, m]).T, y)
        fft_m(y.T, W)
        return y.flatten()
    
    return fft

fft_t12 = fft_two_factor_twiddle(default_config, 3, 4)
fft_t12i = fft_two_factor_twiddle(default_config, 6, 2)

fft_p4 = fft_two_factor(default_config, 2, 2)
fft_t4 = fft_two_factor_twiddle(default_config, 2, 2)

x4 = np.arange(4, dtype='complex64')
x12 = np.arange(12, dtype='complex64')
x15 = np.arange(15, dtype='complex64')

In [58]:
from fftw.pretty_print import ndtotext

In [61]:
print(ndtotext(fft_p12(x12).reshape([3,4])))
print(ndtotext(fft_p12i(x12).reshape([3,4])))
print(ndtotext(np.fft.fft(x12).reshape([3,4]).astype('complex64')))

┌────────────────────────────────────────────────────────────────┐
 [       (66+0j) (-6+22.392303j) (-6+10.392304j)         (-6+6j)] 
 [(-6+3.464101j) (-6+1.6076956j)         (-6+0j) (-6-1.6076956j)] 
 [(-6-3.464101j)         (-6-6j) (-6-10.392304j) (-6-22.392303j)] 
└────────────────────────────────────────────────────────────────┘
┌─────────────────────────────────────────────────────────────────┐
 [        (66+0j) (-6+22.392303j) (-6+10.392304j)         (-6+6j)] 
 [(-6+3.4641016j) (-6+1.6076946j)         (-6+0j) (-6-1.6076946j)] 
 [(-6-3.4641016j)         (-6-6j) (-6-10.392304j) (-6-22.392303j)] 
└─────────────────────────────────────────────────────────────────┘
┌─────────────────────────────────────────────────────────────────┐
 [        (66+0j) (-6+22.392305j) (-6+10.392304j)         (-6+6j)] 
 [(-6+3.4641016j) (-6+1.6076951j)         (-6+0j) (-6-1.6076951j)] 
 [(-6-3.4641016j)         (-6-6j) (-6-10.392304j) (-6-22.392305j)] 
└────────────────────────────────────────────────────

In [62]:
print(ndtotext(fft_t12(x12).reshape([4, 3])))
print(ndtotext(fft_p12i(x12).reshape([2, 6])))
print(ndtotext(fft_t12i(x12).reshape([2, 6])))

┌─────────────────────────────────────────┐
 [(66+0j) (-6+22.392303j) (-6+10.392304j)] 
 [(-6+6j)  (-6+3.464101j) (-6+1.6076956j)] 
 [(-6+0j) (-6-1.6076956j)  (-6-3.464101j)] 
 [(-6-6j) (-6-10.392304j) (-6-22.392303j)] 
└─────────────────────────────────────────┘
┌─────────────────────────────────────────────────────────────────────────────────┐
 [(66+0j) (-6+22.392303j) (-6+10.392304j) (-6+6j) (-6+3.4641016j) (-6+1.6076946j)] 
 [(-6+0j) (-6-1.6076946j) (-6-3.4641016j) (-6-6j) (-6-10.392304j) (-6-22.392303j)] 
└─────────────────────────────────────────────────────────────────────────────────┘
┌─────────────────────────────────────────────────────────────────────────────────┐
 [(66+0j) (-6+22.392303j) (-6+10.392304j) (-6+6j) (-6+3.4641016j) (-6+1.6076946j)] 
 [(-6+0j) (-6-1.6076946j) (-6-3.4641016j) (-6-6j) (-6-10.392304j) (-6-22.392303j)] 
└─────────────────────────────────────────────────────────────────────────────────┘


In [63]:
fft_t4(x4)

array([ 6.+0.j, -2.+2.j, -2.+0.j, -2.-2.j], dtype=complex64)

In [64]:
fft_p4(x4)

array([ 6.+0.j, -2.+2.j, -2.+0.j, -2.-2.j], dtype=complex64)